US Consumer Behaviour 

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
pce = pd.read_csv("../data/PERSONAL_EXPENDITURE.csv")
save = pd.read_csv("../data/SAVING_RATE.csv")
credit = pd.read_csv("../data/CREDIT_OWNED.csv")
cpi = pd.read_csv("https://fred.stlouisfed.org/graph/fredgraph.csv?id=CPIAUCSL")

In [41]:
for df in [credit, save, pce, cpi]:
    df.rename(columns={'observation_date': 'DATE'}, inplace=True)
    df['DATE'] = pd.to_datetime(df['DATE'])

In [42]:
master_df = pce.merge(save, on='DATE', how='inner') \
                .merge(credit, on='DATE', how='inner')\
                .merge(cpi, on='DATE', how='inner')

In [43]:
master_df.rename(columns={
    'PCEC96': 'Expenditure_Billions',
    'PSAVERT': 'Saving_Rate_Pct',
    'TOTALSL': 'Credit_Owned_Billions',
    'CPIAUCSL': 'CPI_Index'
}, inplace=True)

In [44]:
master_df.sort_values('DATE', inplace=True)

In [48]:
master_df['Credit_to_Spend_Ratio'] = master_df['Credit_Owned_Billions'] / master_df['Expenditure_Billions']

master_df['Spend_YoY_Growth'] = master_df['Expenditure_Billions'].pct_change(periods=12) * 100
master_df['Credit_YoY_Growth'] = master_df['Credit_Owned_Billions'].pct_change(periods=12) * 100
master_df['Inflation_YoY'] = master_df['CPI_Index'].pct_change(periods=12) * 100

def assign_regime(date):
    if date < pd.to_datetime('2020-03-01'):
        return '1_Pre_Covid'
    elif date < pd.to_datetime('2021-06-01'):
        return '2_Covid_Stimulus'
    else:
        return '3_Post_Inflation_Shock'

master_df['Regime'] = master_df['DATE'].apply(assign_regime)


master_df.dropna(inplace=True)


print(master_df.sample(10))

          DATE  Expenditure_Billions  Saving_Rate_Pct  Credit_Owned_Billions  \
193 2023-02-01               15491.8              5.5             4884912.07   
154 2019-11-01               14142.8              6.9             4164684.83   
163 2020-08-01               13793.0             13.9             4113990.24   
204 2024-01-01               15793.7              6.4             5003263.54   
138 2018-07-01               13682.7              6.5             3924685.82   
58  2011-11-01               11555.7              6.4             2746799.26   
86  2014-03-01               12108.3              5.2             3150697.02   
107 2015-12-01               12777.3              5.8             3400230.33   
222 2025-07-01               16547.5              4.3             5057212.70   
62  2012-03-01               11660.5              7.6             2791565.95   

     CPI_Index  Credit_to_Spend_Ratio  Spend_YoY_Growth  Credit_YoY_Growth  \
193    301.450             315.322433    